In [298]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [299]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("iamsouravbanerjee/heart-attack-prediction-dataset")

# print("Path to dataset files:", path)

In [300]:
df = pd.read_csv('heart_attack_prediction_dataset.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.shape


(8763, 26)

In [301]:
df.head()

,patient_id,age,sex,cholesterol,blood_pressure,heart_rate,diabetes,family_history,smoking,obesity,...,sedentary_hours_per_day,income,bmi,triglycerides,physical_activity_days_per_week,sleep_hours_per_day,country,continent,hemisphere,heart_attack_risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


Stage: Cleanup and EDA

In [302]:
df.isnull().sum()

patient_id                         0
age                                0
sex                                0
cholesterol                        0
blood_pressure                     0
heart_rate                         0
diabetes                           0
family_history                     0
smoking                            0
obesity                            0
alcohol_consumption                0
exercise_hours_per_week            0
diet                               0
previous_heart_problems            0
medication_use                     0
stress_level                       0
sedentary_hours_per_day            0
income                             0
bmi                                0
triglycerides                      0
physical_activity_days_per_week    0
sleep_hours_per_day                0
country                            0
continent                          0
hemisphere                         0
heart_attack_risk                  0
dtype: int64

No NaN values seen!

In [303]:
df.drop(['patient_id'],inplace=True,axis=1)


See Distribution

In [304]:
print(df['heart_attack_risk'].describe())


count    8763.000000
mean        0.358211
std         0.479502
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: heart_attack_risk, dtype: float64


Feature importnace

In [305]:
df.columns


Index(['age', 'sex', 'cholesterol', 'blood_pressure', 'heart_rate', 'diabetes',
       'family_history', 'smoking', 'obesity', 'alcohol_consumption',
       'exercise_hours_per_week', 'diet', 'previous_heart_problems',
       'medication_use', 'stress_level', 'sedentary_hours_per_day', 'income',
       'bmi', 'triglycerides', 'physical_activity_days_per_week',
       'sleep_hours_per_day', 'country', 'continent', 'hemisphere',
       'heart_attack_risk'],
      dtype='object')

In [306]:
categorical_col = df.select_dtypes(include=['object']).columns
print(categorical_col)

Index(['sex', 'blood_pressure', 'diet', 'country', 'continent', 'hemisphere'], dtype='object')


In [307]:
numerical_col = df.select_dtypes(exclude=['object']).columns
print(numerical_col)

Index(['age', 'cholesterol', 'heart_rate', 'diabetes', 'family_history',
       'smoking', 'obesity', 'alcohol_consumption', 'exercise_hours_per_week',
       'previous_heart_problems', 'medication_use', 'stress_level',
       'sedentary_hours_per_day', 'income', 'bmi', 'triglycerides',
       'physical_activity_days_per_week', 'sleep_hours_per_day',
       'heart_attack_risk'],
      dtype='object')


In [308]:

from sklearn.metrics import mutual_info_score
def mutual_info_heartissue_score(series):
    return mutual_info_score(series, df.heart_attack_risk)
mi = df[categorical_col].apply(mutual_info_heartissue_score)
mi.sort_values(ascending=False).round(6)

blood_pressure    0.286323
country           0.001160
continent         0.000219
hemisphere        0.000081
diet              0.000054
sex               0.000005
dtype: float64

In [309]:
numerical_col = numerical_col.drop("heart_attack_risk")
df[numerical_col].corrwith(df.heart_attack_risk).abs().sort_values(ascending=False)


cholesterol                        0.019340
sleep_hours_per_day                0.018528
diabetes                           0.017225
alcohol_consumption                0.013778
obesity                            0.013318
exercise_hours_per_week            0.011133
triglycerides                      0.010471
income                             0.009628
age                                0.006403
sedentary_hours_per_day            0.005613
physical_activity_days_per_week    0.005014
heart_rate                         0.004251
stress_level                       0.004111
smoking                            0.004051
medication_use                     0.002234
family_history                     0.001652
previous_heart_problems            0.000274
bmi                                0.000020
dtype: float64

In [310]:
#joining_year = ['joining_year']
#df[joining_year].corrwith(df.stars).abs()

Stage: Model Training 

Split data into train validation and test

In [311]:
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
len(df_train), len(df_val), len(df_test)

(5257, 1753, 1753)

In [312]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
y_train = df_train.heart_attack_risk.values
y_val = df_val.heart_attack_risk.values
y_test = df_test.heart_attack_risk.values

del df_train['heart_attack_risk']
del df_val['heart_attack_risk']
del df_test['heart_attack_risk']

One-hot encoding

In [313]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE  # If using SMOTE for class imbalance

dv = DictVectorizer(sparse=False)




#train_dict = df_train[categorical_col + numerical_col].to_dict(orient='records')
train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

test_dict = df_val.to_dict(orient='records')
X_test = dv.transform(test_dict)
#dv.feature_names_

# Step 1: Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Step 2: Handle class imbalance
smote = SMOTE()
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

Training logistic regression

In [314]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
#model = LogisticRegression(solver='liblinear', C=100, max_iter=1, random_state=42, class_weight='balanced')
model = LogisticRegression(class_weight='balanced', penalty='l2', C=0.1, random_state=42, max_iter=1)
model.fit(X_train_res, y_train_res)
y_val_probs = model.predict_proba(X_val_scaled)[:, 1]
auc = roc_auc_score(y_val, y_val_probs)
print(auc)



0.5248839593676691


/home/cpl/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [315]:
def train(df_train, y_train):
    dicts = df_train.to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)
    # Step 1: Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    # Step 2: Handle class imbalance
    smote = SMOTE()
    X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)
    model = LogisticRegression(solver='liblinear', C=1, max_iter=1000, random_state=42)
    model.fit(X_train_res, y_train_res)
    
    return dv, model

def predict(df, dv, model):
    dicts = df.to_dict(orient='records')

    X = dv.transform(dicts)
    X_scaled = scaler.transform(X)
    y_pred = model.predict_proba(X_scaled)[:, 1]

    return y_pred

In [316]:
dv, model = train(df_train, y_train)
y_pred =  predict(df_val, dv, model)
roc_auc_score(y_val, y_pred)

0.5111178905958331

In [317]:
# from sklearn.model_selection import KFold
# #n_splits = [3, 5, 10, 15, 20, 100, 200, 250]
# n_splits = [20]
# for n_split in n_splits: 
#     kfold = KFold(n_splits=n_split, shuffle=True, random_state=1)
#     for train_idx, val_idx in kfold.split(df_full_train):
#         df_train = df_full_train.iloc[train_idx]
#         df_val = df_full_train.iloc[val_idx]

#         y_train = df_train.heart_attack_risk.values
#         y_val = df_val.heart_attack_risk.values

#         dv, model = train(df_train, y_train)
#         y_pred = predict(df_val, dv, model)
        

#     #print(accuracy_score(y_val, y_pred))
#     print(roc_auc_score(y_val, y_pred))

In [318]:
df['heart_attack_risk'].describe()
count = (df['heart_attack_risk'] == 1).sum()
print(count)

3139


Training random forest regressor


In [322]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred = rf.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, y_pred)

0.49019366108891527

In [325]:
max_depth = 10
scores = []
auc_list = []

for s in [1, 3, 5, 10, 50]:
    for n in range(10, 201, 10):
        rf = RandomForestClassifier(n_estimators=n,
                                    max_depth=max_depth,
                                    min_samples_leaf=s,
                                    random_state=1)
        rf.fit(X_train, y_train)

        y_pred = rf.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)

        scores.append((s, n, auc))
        auc_list.append(auc)
print(max(auc_list))        

0.5266364097334826


In [324]:
print(max(scores[2]))        

30


In [185]:
# scores = []
# for n in range(10, 201, 10):
#     rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
#     rf.fit(X_train, y_train)

#     y_pred = rf.predict(X_val)
#     rmse = mean_squared_error(y_val, y_pred, squared=False).round(3)
    
#     scores.append((n, rmse))

In [186]:
# print(scores)

In [187]:
scores = []
for d in [10, 5, 3, 1]:    
    for n in range(10, 201, 10):
        rf = RandomForestRegressor(n_estimators=n, max_depth=d, random_state=1, n_jobs=-1)
        rf.fit(X_train, y_train)

        y_pred = rf.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False).round(3)
        
        scores.append((n, rmse))
        df_scores = pd.DataFrame(scores, columns=['n_estimators', 'rmse'])
    print(df_scores['rmse'].mean())

68.34585
68.72695
69.01266666666668
69.64236249999999


XGBoost

In [188]:
import xgboost as xgb
features = list(dv.get_feature_names_out())
print(len(features))
for feature in features:
    if "[" in feature or "]" in feature or "<" in feature:
        print(feature)
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

5567


In [189]:
xgb_params = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=10)

In [190]:
y_pred = model.predict(dval)
mean_squared_error(y_val, y_pred, squared=False).round(3)

69.05